# Implementation of Random Forests

I implemented Decision Tree algorithm for a previous assignment, now I used that one to implement Random Forests.

Importing necessary modules (Numpy for now)

In [169]:
import numpy as np

Decision Tree (implemented previously by myself!)

In [170]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        # Decision Tree class constructor
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def calculate_entropy(self, y, sample_weight=None):
        # Calculates the entropy of a set of target values
        classes, counts = np.unique(y, return_counts=True)
        if sample_weight is not None:
            probabilities = np.bincount(y, weights=sample_weight) / np.sum(sample_weight)
        else:
            probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))
        return entropy

    def find_best_split(self, X, y, sample_weight):
        # Finds the best split for the decision tree with sample weights
        n_samples, n_features = X.shape
        if n_samples <= self.min_samples_split:
            return None, None

        best_criterion = float('inf')
        best_feature = None
        best_threshold = None

        for feature in range(n_features):
            unique_values = np.unique(X[:, feature])
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2.0

            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                # Information Gain with sample weights
                criterion = (self.calculate_entropy(y[left_mask], sample_weight[left_mask]) *
                             np.sum(left_mask) / n_samples +
                             self.calculate_entropy(y[right_mask], sample_weight[right_mask]) *
                             np.sum(right_mask) / n_samples)

                if criterion < best_criterion:
                    best_criterion = criterion
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def most_common_class(self, y):
        # Finds the most common class in a set of target values
        classes, counts = np.unique(y, return_counts=True)
        most_common_index = np.argmax(counts)
        return classes[most_common_index]

    def build_tree(self, X, y, sample_weight, depth):
        # Recursively builds the decision tree with sample weights
        n_samples, n_features = X.shape
        unique_classes = np.unique(y)

        # Early stopping criteria
        if (len(unique_classes) == 1) or \
           (self.max_depth is not None and depth == self.max_depth) or \
           (n_samples < self.min_samples_split):
            return {'class': self.most_common_class(y)}

        best_feature, best_threshold = self.find_best_split(X, y, sample_weight)

        if best_feature is None:
            return {'class': self.most_common_class(y)}

        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        left_subtree = self.build_tree(X[left_mask], y[left_mask], sample_weight[left_mask], depth + 1)
        right_subtree = self.build_tree(X[right_mask], y[right_mask], sample_weight[right_mask], depth + 1)

        return {'feature': best_feature, 'threshold': best_threshold,
                'left': left_subtree, 'right': right_subtree}

    def fit(self, X, y, sample_weight=None):
        # Fit the decision tree to the training data
        if sample_weight is None:
            sample_weight = np.ones(len(y))
        self.tree = self.build_tree(X, y, sample_weight, depth=0)

    def predict_tree(self, x, node):
        # Recursively predicts the target value for a single input feature
        if 'class' in node:
            return node['class']
        if x[node['feature']] <= node['threshold']:
            return self.predict_tree(x, node['left'])
        else:
            return self.predict_tree(x, node['right'])

    def predict(self, X):
        # Predicts the target values for the input features
        return np.array([self.predict_tree(x, self.tree) for x in X])

## Random Forest:


Random Forests enhance decision tree performance by introducing a key modification that reduces tree correlation. Similar to bagging, multiple decision trees are built on bootstrapped training samples. In constructing these trees, a random subset of predictors ($m$) is considered at each decision point, chosen from the full set of predictors ($p$), with only one predictor from the subset used for the split. This process is repeated for each split, with $m \approx \sqrt{p}$ (can be a different value by setting its hyperparameter `max_features`). By limiting predictor consideration, the algorithm prevents high correlation among trees, improving prediction reliability. Random Forests mitigate bagging's limitation of highly correlated trees, resulting in more varied and reliable predictions. Mathematically, the average predictors considered at each split is $\frac{p - m}{p}$, where $m \approx \sqrt{p}$ (its default value is set be the square root of the number of predictors).

In [171]:
class RandomForest:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, max_features=None):
        # RandomForest class constructor
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features  # Added max_features parameter
        self.trees = []

    def fit(self, X, y):
        # Fit the RandomForest to the training data
        num_features = X.shape[1]
        if self.max_features is None:
            # Default: use all features
            self.max_features = int(np.sqrt(num_features))
        else:
            # Choose a random subset of features (m)
            self.max_features = min(self.max_features, num_features)

        for _ in range(self.n_estimators):
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)

            # Randomly choose a subset of features for this tree
            selected_features = np.random.choice(num_features, self.max_features, replace=False)

            # Use the selected features for training
            tree.fit(X[:, selected_features], y)

            self.trees.append((tree, selected_features))

    def predict(self, X):
        # Predicts the target values for the input features using the ensemble of trees
        predictions = np.array([tree.predict(X[:, selected_features]) for tree, selected_features in self.trees])
        # Take the majority vote as the final prediction
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

## Train/Test on Breast Cancer Dataset

In [172]:
import pandas as pd

In [173]:
with open("Breast_Cancer_dataset.txt", "r") as f:
  bc_file_array = f.read().split("\n")

In [174]:
bc_file_array[0:5]

["'40-49','premeno','15-19','0-2','yes','3','right','left_up','no','recurrence-events'",
 "'50-59','ge40','15-19','0-2','no','1','right','central','no','no-recurrence-events'",
 "'50-59','ge40','35-39','0-2','no','2','left','left_low','no','recurrence-events'",
 "'40-49','premeno','35-39','0-2','yes','3','right','left_low','yes','no-recurrence-events'",
 "'40-49','premeno','30-34','3-5','yes','2','left','right_up','no','recurrence-events'"]

## Pre-process the dataset

In [175]:
# Define attribute names
attributes = ['age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
              'deg-malig', 'breast', 'breast-quad', 'irradiat', 'Class']

# Define categorical values for encoding
categorical_values = {
    'age': ['10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-99'],
    'menopause': ['lt40','ge40','premeno'],
    'tumor-size': ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59'],
    'inv-nodes': ['0-2','3-5','6-8','9-11','12-14','15-17','18-20','21-23','24-26','27-29','30-32','33-35','36-39'],
    'node-caps': ['yes','no'],
    'deg-malig': ['1','2','3'],
    'breast': ['left','right'],
    'breast-quad': ['left_up','left_low','right_up','right_low','central'],
    'irradiat': ['yes','no'],
    'Class': ['no-recurrence-events','recurrence-events']
}

# Create an empty list to store DataFrames
dfs = []

# Process each line of the dataset
for line in bc_file_array:
    values = line.split(',')
    df = pd.DataFrame([dict(zip(attributes, values))])
    dfs.append(df)

# Concatenate the DataFrames
df = pd.concat(dfs, ignore_index=True)

In [176]:
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,'40-49','premeno','15-19','0-2','yes','3','right','left_up','no','recurrence-events'
1,'50-59','ge40','15-19','0-2','no','1','right','central','no','no-recurrence-events'
2,'50-59','ge40','35-39','0-2','no','2','left','left_low','no','recurrence-events'
3,'40-49','premeno','35-39','0-2','yes','3','right','left_low','yes','no-recurrence-events'
4,'40-49','premeno','30-34','3-5','yes','2','left','right_up','no','recurrence-events'


### Handling missing (null) values by dropping their associated rows

In [177]:
df = df.replace('?', pd.NA)
df = df.dropna()
df = df.reset_index(drop=True)

In [178]:
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,'40-49','premeno','15-19','0-2','yes','3','right','left_up','no','recurrence-events'
1,'50-59','ge40','15-19','0-2','no','1','right','central','no','no-recurrence-events'
2,'50-59','ge40','35-39','0-2','no','2','left','left_low','no','recurrence-events'
3,'40-49','premeno','35-39','0-2','yes','3','right','left_low','yes','no-recurrence-events'
4,'40-49','premeno','30-34','3-5','yes','2','left','right_up','no','recurrence-events'


## Encoding categorical features

- One-Hot Encoding:

For binary categorical features such as `node-caps`, `breast`, and `irradiat`, one-hot encoding is an appropriate method. This technique involves creating binary columns for each category.

- Ordinal Encoding:

Features like `age`, `menopause`, `tumor-size`, `inv-nodes`, and `deg-malig` exhibit a natural order. To represent these features numerically based on their order, ordinal encoding is a suitable approach.

- Label Encoding:

In the case of `breast-quad` and `Class`, which possesses more than two categories without a distinct ordinal relationship, label encoding can be used. This method assigns a unique integer to each category within the feature.

In [179]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Define features for each encoding method
one_hot_features = ['node-caps', 'breast', 'irradiat']
ordinal_features = ['age', 'menopause', 'tumor-size', 'inv-nodes', 'deg-malig']
label_features = ['breast-quad', 'Class']

# One-Hot Encoding
df = pd.get_dummies(df, columns=one_hot_features)

# Ordinal Encoding
ordinal_encoder = OrdinalEncoder()
df[ordinal_features] = ordinal_encoder.fit_transform(df[ordinal_features])

# Label Encoding
label_encoder = LabelEncoder()
for feature in label_features:
  df[feature] = label_encoder.fit_transform(df[feature])


In [180]:
df.head()

,age,menopause,tumor-size,inv-nodes,deg-malig,breast-quad,Class,node-caps_'no',node-caps_'yes',breast_'left',breast_'right',irradiat_'no',irradiat_'yes'
0,2.0,2.0,2.0,0.0,2.0,2,1,0,1,0,1,1,0
1,3.0,0.0,2.0,0.0,0.0,0,0,1,0,0,1,1,0
2,3.0,0.0,6.0,0.0,1.0,1,1,1,0,1,0,1,0
3,2.0,2.0,6.0,0.0,2.0,1,0,0,1,0,1,0,1
4,2.0,2.0,5.0,4.0,1.0,4,1,0,1,1,0,1,0


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              277 non-null    float64
 1   menopause        277 non-null    float64
 2   tumor-size       277 non-null    float64
 3   inv-nodes        277 non-null    float64
 4   deg-malig        277 non-null    float64
 5   breast-quad      277 non-null    int64  
 6   Class            277 non-null    int64  
 7   node-caps_'no'   277 non-null    uint8  
 8   node-caps_'yes'  277 non-null    uint8  
 9   breast_'left'    277 non-null    uint8  
 10  breast_'right'   277 non-null    uint8  
 11  irradiat_'no'    277 non-null    uint8  
 12  irradiat_'yes'   277 non-null    uint8  
dtypes: float64(5), int64(2), uint8(6)
memory usage: 16.9 KB


# Splitting the dataset to train/test

In [182]:
from sklearn.model_selection import train_test_split

X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (193, 12)
X_test shape: (84, 12)
y_train shape: (193,)
y_test shape: (84,)


In [183]:
from sklearn.metrics import accuracy_score

# Converting the dataframes into Numpy Arrays
X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values

### Running the **Decision Tree Classifier** on Breast Cancer Dataset:

In [184]:
dt = DecisionTree(max_depth = 8)
dt.fit(X_train, y_train)
predictions = dt.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.64


### Running the **Random Forest Classifier** on Breast Cancer Dataset:

In [185]:
dt = RandomForest(max_depth = 8)
dt.fit(X_train, y_train)
predictions = dt.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.73


**A major improvement can be seen when using Random Forests over Decision Tree!**

--------------------------------------------------------------------------------

# Implementation of AdaBoost Algorithm

AdaBoost is a type of ensemble learning that merges weak learners by giving importance to data points and highlighting incorrectly classified samples during iterative training. It adjusts the weights of samples and classifiers through mathematical calculations to create the ultimate prediction.


At each iteration, AdaBoost fits a weak classifier $h_t(x): \mathbb{X} \rightarrow \{-1, +1\}$ and computes the error $\varepsilon_t$:

$$
\varepsilon_t = \sum_{i=1}^{N} w_{i,t} \cdot \mathbb{1}(h_t(x_i) \neq y_i)
$$


The weight of the weak classifier is then calculated as:

$$
\alpha_t = \frac{1}{2} \ln\left(\frac{1 - \varepsilon_t}{\varepsilon_t}\right)
$$

The weights of misclassified samples are updated as:

$$
w_{i,t+1} = w_{i,t} \cdot \exp(-\alpha_t \cdot y_i \cdot h_t(x_i))
$$

The ensemble prediction is given by:
$$
H(x) = \text{sign}\left(\sum_{t=1}^{T} \alpha_t \cdot h_t(x)\right)
$$

where $T$ is the number of weak classifiers.


In [186]:

class AdaBoost:
    def __init__(self, num_classifiers=5, learning_rate=0.1, max_depth=1):
        self.num_classifiers = num_classifiers
        self.learning_rate = learning_rate
        self.max_depth = max_depth

    def fit(self, X_train, y_train):
        num_samples, num_features = X_train.shape

        # Initialize weights
        sample_weights = np.full(num_samples, 1/num_samples)

        # Initialize models and model weights
        self.classifiers = []
        self.classifier_weights = []

        for _ in range(self.num_classifiers):
            # Fit a classifier
            classifier = DecisionTree(max_depth=self.max_depth)
            classifier.fit(X_train, y_train, sample_weight=sample_weights)

            # Compute error
            y_pred = classifier.predict(X_train)
            error = sample_weights[(y_pred != y_train)].sum() / sample_weights.sum()

            # Compute model weight
            classifier_weight = self.learning_rate * (np.log((1 - error) / error) + np.log(num_features - 1))

            # Update weights
            sample_weights[y_pred != y_train] *= np.exp(classifier_weight)

            # Normalize weights
            sample_weights /= sample_weights.sum()

            # Save model and model weight
            self.classifiers.append(classifier)
            self.classifier_weights.append(classifier_weight)

    def predict(self, X_test):
        # Compute predictions for each model
        predictions = np.array([classifier.predict(X_test) for classifier in self.classifiers])

        # Compute final predictions
        final_prediction = np.sign(np.dot(self.classifier_weights, predictions))

        return final_prediction

In [187]:
ab = AdaBoost()
ab.fit(X_train, y_train)
predictions = ab.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.63


# Comparison between Decision Tree, Random Forest and AdaBoost on Breast Cancer Dataset!

In [189]:
import pandas as pd

results = {
    'Model': ['Decision Tree', 'Random Forest', 'AdaBoost'],
    'Accuracy': [0.64, 0.74, 0.63]
}

df = pd.DataFrame(results)
df

,Model,Accuracy
0,Decision Tree,0.64
1,Random Forest,0.74
2,AdaBoost,0.63
